# Detecting and deleting book/article titles


Setting up the text for regex


**Download txt file to preprocessing folder**

**Run pip install ipysheet**

In [109]:
import re
import pandas as pd
from ipysheet import from_dataframe, to_dataframe

#Replace file name assigned to book (make sure text file saved to preprocessing folder)


book = "gendertrouble-text.txt"


booktext=open(book,"r", encoding="utf8")


test_str = booktext.read()

## Round 1: auto deletion of high accuracy title matching sequences

In [ ]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility


regex = r'“[^”[:lower:]]+”[^“”]+Vol\.|“[^”[:lower:]]+”[^“”]+No\.|eds\.[^V()]+Vol\.|eds\.[^()V]+\([^)]+\)|“[^”[:lower:]]+”[^()]+\([^()]+\)|\.,[^()]+\([^)]+\)|:[^()]+\([^()]+\)|“[^.]+pp\.|, “[^.]+p\.|“[^.]+ p\. \d|“[^V\d]+Vol\. \d|“[^(\d]+\([^\d]+\d\d\d\d\)'


subst = ""


# You can manually specify the number of replacements by changing the 4th argument
result = re.sub(regex, subst, test_str, 0, re.MULTILINE)


if result:
    test_str = result
# Note: for Python 2.7 compatibility, use ur"" to prefix the regex and u"" to prefix the test string and substitution.

## Round 2: Cycling through general regex with user input confirming deletion of a portion of text

This section is still in progress as not as effective in finding possible matches as I would want. Current alternative is only having user run Round 1 and then manually delete the remaining matches.

In [101]:
# Sets up a panda dataframe of identified text for user to go through

regex_match_list = ['“[^”]+”[^()]+\([^()]+\)','\.,[^()]+\([^)]+\)',':[^()]+\([^()]+\)',',[^()]+\([^)]+\)',':[^()]+\([^()]+\)']

prematches = []

matches = []

# Goes expression by expression to find possible matches

for reg in regex_match_list:
    regex = '((.*(\n|\r|\r\n)){2})' + reg + '((.*(\n|\r|\r\n)){2})'
    
    prematches = prematches + re.findall(regex, test_str) 

for m in prematches:
    #matches.append(r"'"+''.join(m)+"'")
    matches.append(repr(''.join(m)))


⚠️ Note: editing the spreadsheet may cause it to collapse. Don't worry just look below the following block of code and click the three dots following "Outputs are collapsed". ⚠️

In [ ]:
# Creates an interactive spreadsheet where user copy pastes actual matches on the cell 1 over.

possible_matches = pd.DataFrame({"Possible Matches": matches, "Correct Matches": [None for i in range(len(matches))]})

sheet = from_dataframe(possible_matches)

sheet.column_width = [10,10,5,5,5,10] #adjust column widths so user does not have to scroll horizontally

sheet

## Processing correct matches given by user

In [87]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility

correct_matches = ""

new_possible_matches = to_dataframe(sheet)

for m in new_possible_matches["Correct Matches"]:
    if m != None:
        if len(correct_matches) != 0:
            correct_matches = correct_matches + "|" + m
        else:
            correct_matches = m

regex = correct_matches

subst = ""

# You can manually specify the number of replacements by changing the 4th argument
result = re.sub(regex, subst, test_str, 0, re.MULTILINE)

if result:
    test_str = result

# Note: for Python 2.7 compatibility, use ur"" to prefix the regex and u"" to prefix the test string and substitution.

## Round 3: Output finalized txt file of cleaned up text
    Saves a new txt file named 'given book file name - cleaned .txt'

In [95]:
cleaned_book = book.replace(".txt","")+"-cleaned"+".txt"

text_file = open(cleaned_book, "wt", encoding="utf-8")
n = text_file.write(test_str)
text_file.close()